In [1]:
import warnings
warnings.filterwarnings("ignore")

from os.path import join
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import missingno as msno

data_dir = '~/00_Main/daton/220926/datathon_A_Team'
ext_data_dir = '~/00_Main/daton/220926/datathon_A_Team/ext_data'

train_path = join(data_dir, 'train.csv')
train = pd.read_csv(train_path, encoding='UTF-8')

test_path = join(data_dir, 'test.csv')
test = pd.read_csv(test_path, encoding='UTF-8')

day_care_center_path = join(data_dir, 'day_care_center.csv')
day_care_center = pd.read_csv(day_care_center_path, encoding='UTF-8')

park_path = join(data_dir, 'park.csv')
park = pd.read_csv(park_path, encoding='UTF-8')

In [2]:
# train_len 저장하고 데이터 결합

train_len = len(train)

data = pd.concat([train, test], axis=0)

In [3]:
    # 구 추가하기
busan_dong_path = join(ext_data_dir, 'busan.csv')
seoul_dong_path = join(ext_data_dir, 'seoul.csv')

busan_dong = pd.read_csv(busan_dong_path, encoding='cp949')
seoul_dong = pd.read_csv(seoul_dong_path, encoding='cp949')

In [4]:
seoul_gu = seoul_dong['법정동명'].groupby(seoul_dong['시군구명']).unique()
seoul_gu = dict(seoul_gu)

seoul_gu.keys()

dict_keys(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'])

In [5]:
busan_gu = busan_dong['법정동명'].groupby(busan_dong['시군구명']).unique()
busan_gu = dict(busan_gu)

busan_gu.keys()

dict_keys(['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구'])

In [6]:
seoul_gu.keys() & busan_gu.keys()

{'강서구', '중구'}

In [7]:
seoul_gu = {'서울'+k: '서울'+v for k, v in seoul_gu.items()}
busan_gu = {'부산'+k: '부산'+v for k, v in busan_gu.items()}

In [8]:
seoul_gu.update(busan_gu)
gu = seoul_gu

In [9]:
data['city'] = data['city'].apply(lambda x: x[:2])
data['dong'] = data['city'] + data['dong']

data.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울,서울신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500.0
1,1,5399,서울,서울필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000.0
2,2,3578,서울,서울필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500.0
3,3,10957,서울,서울내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000.0
4,4,10639,서울,서울내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000.0


In [10]:
for k, v in gu.items():
    data.loc[data['dong'].isin(v), 'gu'] = k

In [11]:
data['gu'].isna().sum()

0

In [12]:
grouped = data['transaction_real_price'].groupby(by=data['gu']).mean()
grouped

gu
부산강서구     31240.298350
부산금정구     22736.463471
부산기장군     19655.473365
부산남구      23469.688425
부산동구      18717.596677
부산동래구     23279.718287
부산부산진구    18753.698471
부산북구      17981.371051
부산사상구     14933.562957
부산사하구     15141.810976
부산서구      19592.257367
부산수영구     26188.155107
부산연제구     24178.487135
부산영도구     14159.843340
부산중구      13321.269319
부산해운대구    26907.117468
서울강동구     41080.281168
서울관악구     42953.450617
서울금천구     26812.244082
서울동대문구    37553.361541
서울동작구     28930.000000
서울마포구     55152.987777
서울서대문구    41020.762013
서울성동구     66043.581737
서울성북구     31381.920270
서울송파구     69447.390203
서울영등포구    49252.175420
서울용산구     80325.633789
서울은평구     38371.311883
서울종로구     48968.885914
서울중구      54279.653664
서울중랑구     30611.301468
Name: transaction_real_price, dtype: float64

In [13]:
grouped = data['transaction_real_price'].groupby(by=data['dong']).mean()
dong_price = grouped.sort_values(ascending=False)

for i, k in enumerate(dong_price.index):
    data.loc[data['dong'] == k, 'dong'] = i

In [14]:
data['dong'] = data['dong'].astype(int)

In [15]:
grouped = data['transaction_real_price'].groupby(data['city']).mean()
grouped

data['city'] = data['city'].apply(lambda x: 1 if x == '서울' else 0)

In [16]:
regex = lambda x: re.sub('\(.*\)|\s-\s.*', '', x)
data['apt'] = data['apt'].apply(regex)

data['apt']

0            신현
1        사직파크맨션
2        두레엘리시안
3         파크팰리스
4          킹스매너
         ...   
5458        위너스
5459    래미안퍼스티지
5460      삼성래미안
5461       동양드림
5462     센텀그린타워
Name: apt, Length: 1222016, dtype: object

In [17]:
grouped = data['transaction_real_price'].groupby(by=data['apt']).mean()
apt_price_top100 = grouped.sort_values(ascending=False)[:1000]

apt_price_top100

apt
상지리츠빌카일룸2차     530000.000000
트라움하우스3        490000.000000
아펠바움           465860.500000
상지리츠빌카일룸3차     427125.000000
삼성동상지리츠빌카일룸    418992.666667
                   ...      
강남한신휴플러스6단지     69235.200000
우성4차            69220.270270
현대레이크빌1         69220.000000
가락삼익맨숀          69214.256684
힐튼빌리지2차         69187.500000
Name: transaction_real_price, Length: 1000, dtype: float64

In [18]:
grouped = data['transaction_real_price'].groupby(by=data['apt']).count()
apt_count_top100 = grouped.sort_values(ascending=False)[:1000]

apt_count_top100

apt
현대         17716
한신         10135
삼성          6771
대우          6390
신동아         6386
           ...  
문현삼성힐타워      298
왕자           297
성지           297
위례24단지       297
국제산장         297
Name: transaction_real_price, Length: 1000, dtype: int64

In [19]:
most_name = (set(apt_price_top100.reset_index()['apt']) & set(apt_count_top100.reset_index()['apt']))
most_name


{'2차한양아파트',
 'DMC파크뷰자이1단지',
 'e-편한세상',
 '가락삼익맨숀',
 '강서힐스테이트',
 '강촌',
 '개포주공 1단지',
 '개포주공 3단지',
 '개포주공 4단지',
 '개포주공 5단지',
 '개포주공 6단지',
 '개포주공 7단지',
 '고덕 아이파크',
 '광화문풍림스페이스본',
 '대림e-편한세상4',
 '대치삼성',
 '대치아이파크',
 '대치현대',
 '더샵스타시티',
 '도곡렉슬',
 '동부센트레빌',
 '둔촌주공2단지',
 '둔촌주공3단지',
 '둔촌주공4단지',
 '래미안 옥수 리버젠',
 '래미안그레이튼',
 '래미안파크팰리스',
 '래미안퍼스티지',
 '레이크팰리스',
 '롯데캐슬갤럭시',
 '롯데캐슬클래식',
 '롯데캐슬프레미어',
 '리센츠',
 '메세나폴리스',
 '목동신시가지1',
 '목동신시가지13',
 '목동신시가지2',
 '목동신시가지3',
 '목동신시가지5',
 '목동신시가지7',
 '목동신시가지9',
 '문래자이',
 '문정래미안',
 '미성2차',
 '반포 주공1단지',
 '반포경남아파트',
 '반포리체',
 '반포미도',
 '반포자이',
 '방배1차현대홈타운',
 '삼부',
 '삼성동힐스테이트 1단지',
 '삼풍아파트',
 '상도엠코타운애스톤파크',
 '상록수',
 '상암월드컵파크4단지',
 '서초래미안',
 '송파삼성래미안아파트',
 '시범',
 '신동아1',
 '신반포2',
 '신반포4',
 '신반포8',
 '아이파크',
 '에이아이디차관주택',
 '역삼래미안',
 '역삼푸르지오',
 '올림픽선수기자촌1단지',
 '올림픽선수기자촌2단지',
 '올림픽선수기자촌3단지',
 '우성아파트',
 '은마',
 '이수힐스테이트',
 '잠실엘스',
 '장미1',
 '장미2',
 '주공아파트 5단지',
 '진주',
 '진흥',
 '청구e편한세상',
 '청담삼익아파트',
 '타워팰리스1',
 '타워팰리스2',
 '트리지움',
 '파크리오',
 '한가람',
 '한강',
 '한남더힐',
 '한보미도맨션1',
 '한

In [20]:
# 토큰화
for name in most_name:
    data[f'{name}'] = data['apt'].str.contains(name).astype('int')

In [21]:
data.drop(columns=['apt', 'addr_kr', 'jibun'], inplace=True)

In [22]:
grouped = data['transaction_real_price'].groupby(by=data['city']).mean()
apt_price_top100 = grouped.sort_values(ascending=False)[:100]

apt_price_top100

city
1    49169.779317
0    21102.045856
Name: transaction_real_price, dtype: float64

In [23]:
grouped = data['transaction_real_price'].groupby(by=data['gu']).mean()
apt_price_top100 = grouped.sort_values(ascending=False)[:100]

apt_price_top100

gu
서울용산구     80325.633789
서울송파구     69447.390203
서울성동구     66043.581737
서울마포구     55152.987777
서울중구      54279.653664
서울영등포구    49252.175420
서울종로구     48968.885914
서울관악구     42953.450617
서울강동구     41080.281168
서울서대문구    41020.762013
서울은평구     38371.311883
서울동대문구    37553.361541
서울성북구     31381.920270
부산강서구     31240.298350
서울중랑구     30611.301468
서울동작구     28930.000000
부산해운대구    26907.117468
서울금천구     26812.244082
부산수영구     26188.155107
부산연제구     24178.487135
부산남구      23469.688425
부산동래구     23279.718287
부산금정구     22736.463471
부산기장군     19655.473365
부산서구      19592.257367
부산부산진구    18753.698471
부산동구      18717.596677
부산북구      17981.371051
부산사하구     15141.810976
부산사상구     14933.562957
부산영도구     14159.843340
부산중구      13321.269319
Name: transaction_real_price, dtype: float64

In [24]:
grouped = data['transaction_real_price'].groupby(by=data['dong']).mean()
apt_price_top100 = grouped.sort_values(ascending=False)[:100]

apt_price_top100

dong
0     269888.888889
1     164534.722914
2     161403.700000
3     153497.331633
4     139906.140351
          ...      
95     53131.640288
96     53113.386275
97     52965.720034
98     52300.889831
99     51930.730435
Name: transaction_real_price, Length: 100, dtype: float64

In [25]:
train_sample = data[:train_len]

In [26]:
train_sample['transaction_year_month'] = train_sample['transaction_year_month'].astype(str)

In [27]:
train_year = train_sample.copy()
train_month = train_sample.copy()

In [28]:
train_year['transaction_year_month'] = train_sample['transaction_year_month'].apply(lambda i:i[:-2])
train_month['transaction_year_month'] = train_sample['transaction_year_month'].apply(lambda i:i[2:])

In [29]:
seoul_year = train_year[train_year['city'] == 1]
busan_year = train_year[train_year['city'] == 0]
seoul_month = train_month[train_month['city'] == 1]
busan_month = train_month[train_month['city'] == 0]

In [30]:
train = data[:train_len]
test = data[train_len:]

In [31]:
grouped = data['transaction_real_price'].groupby(by=data['gu']).mean()

gu_price = grouped.sort_values(ascending=False)

for i, k in enumerate(gu_price.index):
    data.loc[data['gu'] == k, 'gu'] = i

In [32]:
data['gu'] = data['gu'].astype(int)

# 데이터 정규화

In [33]:
del data['transaction_date']

In [34]:
t = data['transaction_year_month'].min()//100
data['transaction_year_month'] = data['transaction_year_month'].apply(lambda x: ((x//100) - t) * 12 + (x%100))

In [35]:
data['floor'] += 5

In [36]:
log_columns = ['apartment_id', 'dong', 'exclusive_use_area', 'year_of_completion', 'transaction_year_month', 'floor', 'gu', 'transaction_real_price']

for i in log_columns:
    data[i] = np.log1p(data[i].values)

In [37]:
train = data[:train_len] 
test = data[train_len:]

In [38]:
train_id = train['transaction_id']
price = train['transaction_real_price']

del train['transaction_real_price']
del train['transaction_id']

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [40]:
x_train, x_test, y_train, y_test = train_test_split(train,
                                                    price,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [41]:
# 손실함수 MSE 함수 정의

def MSE(a,b):
    mse = ((a-b)**2).mean() # 두 값의 차이의 제곱의 평균
    return mse

# 손실함수 RMSE 함수 정의

def RMSE(a, b):
    mse = ((a - b) ** 2).mean() # 두 값의 차이의 제곱의 평균
    rmse = mse ** 0.5 # mse의 제곱근
    return rmse

In [42]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42, n_jobs = -1)

In [44]:
model = RandomForestRegressor(random_state=42, n_jobs = -1)
model.fit(x_train, y_train)
R_pred = model.predict(test)

In [82]:
del test['transaction_real_price']

In [106]:
R_pred = model.predict(x_test)
sub_pred = model.predict(test)

In [45]:
y_label = np.expm1(y_test)
prediction = np.expm1(R_pred)

In [108]:
R_pred = np.expm1(R_pred)

In [96]:
y_label

1098039     31000.0
432251      30200.0
565937      91500.0
448502      23000.0
439435      22000.0
             ...   
936587      16150.0
879661      60500.0
618015      52500.0
947368      21000.0
1172910    141500.0
Name: transaction_real_price, Length: 243311, dtype: float64

In [100]:
prediction

array([36011.97386516, 12903.8161785 , 13326.42612094, ...,
       58302.47524771, 47160.00325618, 15552.67863141])

In [102]:
y_label.shape

(243311,)

In [104]:
y_label

1098039     31000.0
432251      30200.0
565937      91500.0
448502      23000.0
439435      22000.0
             ...   
936587      16150.0
879661      60500.0
618015      52500.0
947368      21000.0
1172910    141500.0
Name: transaction_real_price, Length: 243311, dtype: float64

In [109]:
RMSE(y_label, R_pred)

3886.908320604547

In [103]:
prediction.shape

(5463,)

In [114]:
submission = pd.read_csv('~/00_Main/daton/220926/datathon_A_Team/submission.csv')
submission.head()

,transaction_id,transaction_real_price
0,1145756,0
1,1198704,0
2,1222384,0
3,1179897,0
4,1223091,0


In [115]:
submission['transaction_real_price'] = prediction

In [116]:
submission.head()

,transaction_id,transaction_real_price
0,1145756,36011.973865
1,1198704,12903.816178
2,1222384,13326.426121
3,1179897,114123.139864
4,1223091,46167.571987


In [119]:
submission.to_csv('~/00_Main/daton/220926/datathon_A_Team/practice.csv', index=False)